In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glob import glob
train_dir = '/home/jupyter/datasets/xview/train'

In [3]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn

In [ ]:
import albumentations as al

augment = al.Compose([
        al.HorizontalFlip(p=.5),
        al.VerticalFlip(p=.5),
        al.RandomRotate90(p=.5),
        al.Transpose(p=.5),
        al.GridDistortion(p=.5, distort_limit=.2),
        al.ShiftScaleRotate(p=.5),
        al.RandomBrightnessContrast(p=.5)
])

In [ ]:
from xv.nn import dataset
from xv import util

train_dataset = dataset.XViewSegmentationDataset(path=train_dir, resolution=(1024,1024), augment=augment)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=10,
    shuffle=True,
    num_workers=12,
    pin_memory=True
)

In [ ]:
from xv.nn.losses import loss_dict, WeightedLoss

loss_weights = {
    #'bcewithlogits': 1.,
    'jaccard': 1.,
    #'dice':1,
    'focal':1
}

loss = WeightedLoss({loss_dict[l]():w for l, w in loss_weights.items()})

In [ ]:
from xv.nn.solaris.model_io import get_model
model = get_model('xdxd_spacenet4', 'torch')

In [ ]:
ix = 1000
i = train_dataset[ix]
images, masks = i['images'], i['masks']
image = images['post']
image = np.array(train_dataset.inverse_transform_image(image))

util.vis_im_mask(image, masks['damage'], size=(512*2,512*2), opacity=.3);